In [1]:
"""Import all the relevant libraries."""

from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from sensor_transaction import SensorTransaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from cluster import Cluster
import kdtree

In [2]:
"""Initialize Nodes with Genesis Block"""

print('START')
test_num = 1
MAX_NODES = 9999
num_nodes = 10
num_clusters = 16
forging_interval = 10  # Time interval.  10 units of time
nodes = [] #list of all nodes
blockchain_dimensions = 4
genesis_node_id = int(num_nodes/2)

#itterate through and initialize each node
then = time.time()
for i in range(num_nodes):
    new_node = Node(test_num, i+1, 0)
    #new_node.blockchain = mkd_blockchain
    new_node.start_listener(f'{test_num}.c0')
    nodes.append(new_node)
now = time.time()-then
print(f'block init time @ {num_nodes} nodes: {now}')

#  initialize each cluster
clusters = []
for i in range(num_clusters):
    new_cluster = Cluster(i+1, test_num)
    new_cluster.start_listener(f'{test_num}.c{i+1}')
    clusters.append(new_cluster)
    print(f'cluster {new_cluster.cluster_id} initialized')

genesis_forger = nodes[genesis_node_id-1].wallet.public_key_string()
mkd_blockchain = MKDBlockchain(blockchain_dimensions, genesis_node_id, genesis_forger)


publisher = nodes[genesis_node_id-1]

start_broadcast = time.time()
publisher.publish(mkd_blockchain)
completed_broadcast = time.time() - start_broadcast
print(f'time to complete broadcast to {num_nodes} nodes: {completed_broadcast}')

START
block init time @ 10 nodes: 19.885000705718994
cluster 1 initialized
cluster 2 initialized
cluster 3 initialized
cluster 4 initialized
cluster 5 initialized
cluster 6 initialized
cluster 7 initialized
cluster 8 initialized
cluster 9 initialized
cluster 10 initialized
cluster 11 initialized
cluster 12 initialized
cluster 13 initialized
cluster 14 initialized
cluster 15 initialized
cluster 16 initialized
time to complete broadcast to 10 nodes: 0.0010008811950683594


In [3]:
"""Attempting to add a second block to the kdtree_blockchain"""
number_of_forges = 100

for i in range(number_of_forges):
    random_node = random.randint(0, num_nodes-1)
    forge_node = nodes[random_node]
    forge_node.mkd_forge()

for node in nodes:
    print(f'Node_id: {node.node_id}')
    print(f'Tree size: {node.blockchain_size}')
    print(f'Left size: {node.blockchain.blocks.left_size}')
    print(f'Right size: {node.blockchain.blocks.right_size}')

Node_id: 1
Tree size: 101
Left size: 41
Right size: 59
Node_id: 2
Tree size: 101
Left size: 41
Right size: 59
Node_id: 3
Tree size: 101
Left size: 41
Right size: 59
Node_id: 4
Tree size: 101
Left size: 41
Right size: 59
Node_id: 5
Tree size: 101
Left size: 41
Right size: 59
Node_id: 6
Tree size: 101
Left size: 41
Right size: 59
Node_id: 7
Tree size: 101
Left size: 41
Right size: 59
Node_id: 8
Tree size: 101
Left size: 41
Right size: 59
Node_id: 9
Tree size: 101
Left size: 41
Right size: 59
Node_id: 10
Tree size: 101
Left size: 41
Right size: 59
